<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Ejemplo de datos para los sitios de lanzamiento (deberías sustituir esto con tus datos reales)
datos_sitios_lanzamiento = {
    'Nombre_Sitio': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitud': [28.562302, 28.563197, 28.573255, 34.632834],
    'Longitud': [-80.577356, -80.576820, -80.646895, -120.610745]
}

# Crear un DataFrame con los datos
df_sitios = pd.DataFrame(datos_sitios_lanzamiento)

# Crear el mapa centrado en una ubicación central
mapa = folium.Map(location=[df_sitios['Latitud'].mean(), df_sitios['Longitud'].mean()], zoom_start=5)

# Añadir un Cluster de Marcadores
marker_cluster = MarkerCluster().add_to(mapa)

# Marcar todos los sitios de lanzamiento en el mapa
for idx, row in df_sitios.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        popup=row['Nombre_Sitio'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

# Mostrar el mapa
mapa


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Datos de ejemplo para los sitios de lanzamiento
datos_sitios_lanzamiento = {
    'Nombre_Sitio': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitud': [28.562302, 28.563197, 28.573255, 34.632834],
    'Longitud': [-80.577356, -80.576820, -80.646895, -120.610745]
}

# Crear el DataFrame con los datos
df_sitios = pd.DataFrame(datos_sitios_lanzamiento)

# Crear el mapa centrado en la ubicación promedio de los sitios
mapa = folium.Map(location=[df_sitios['Latitud'].mean(), df_sitios['Longitud'].mean()], zoom_start=5)

# Añadir un MarkerCluster para los marcadores
marker_cluster = MarkerCluster().add_to(mapa)

# Agregar un círculo y un marcador para cada sitio de lanzamiento
for idx, row in df_sitios.iterrows():
    # Agregar un círculo alrededor de cada sitio de lanzamiento
    folium.Circle(
        location=[row['Latitud'], row['Longitud']],
        radius=1000,  # Radio en metros
        color='#000000',  # Color del borde del círculo
        fill=True,  # Rellenar el círculo
        fill_color='#000000',  # Color de relleno
        fill_opacity=0.4  # Opacidad del relleno
    ).add_to(mapa)

    # Agregar un marcador con un popup y un icono personalizado
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['Nombre_Sitio']
        ),
        popup=row['Nombre_Sitio']
    ).add_to(marker_cluster)

# Mostrar el mapa
mapa


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Ejemplo de datos de sitios y resultados de lanzamientos
datos_lanzamientos = {
    'Nombre_Sitio': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Latitud': [28.562302, 28.563197, 28.573255, 34.632834],
    'Longitud': [-80.577356, -80.576820, -80.646895, -120.610745],
    'Mission_Outcome': ['Success', 'Failure', 'Success', 'Unknown']  # Resultado de la misión
}

# Crear el DataFrame con los datos
df_lanzamientos = pd.DataFrame(datos_lanzamientos)

# Crear el mapa centrado en la ubicación promedio de los sitios
mapa = folium.Map(location=[df_lanzamientos['Latitud'].mean(), df_lanzamientos['Longitud'].mean()], zoom_start=5)

# Añadir un MarkerCluster para los marcadores
marker_cluster = MarkerCluster().add_to(mapa)

# Agregar un círculo y un marcador para cada sitio de lanzamiento
for idx, row in df_lanzamientos.iterrows():
    # Definir el color del círculo y el mensaje según el resultado de la misión
    if row['Mission_Outcome'] == 'Success':
        color = '#27ae60'  # Verde para éxito
        mensaje = '¡Lanzamiento Exitoso!'
    elif row['Mission_Outcome'] == 'Failure':
        color = '#e74c3c'  # Rojo para fallo
        mensaje = 'Lanzamiento Fallido'
    else:
        color = '#95a5a6'  # Gris si no se especifica resultado
        mensaje = 'Resultado Desconocido'

    # Agregar un círculo alrededor de cada sitio de lanzamiento
    folium.Circle(
        location=[row['Latitud'], row['Longitud']],
        radius=1000,  # Radio en metros
        color=color,  # Color del borde del círculo
        fill=True,  # Rellenar el círculo
        fill_color=color,  # Color de relleno
        fill_opacity=0.4  # Opacidad del relleno
    ).add_to(mapa)

    # Agregar un marcador con un popup y un icono personalizado
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['Nombre_Sitio']
        ),
        popup=mensaje  # Popup con el mensaje del resultado de la misión
    ).add_to(marker_cluster)

# Mostrar el mapa
mapa


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()
print(spacex_df.columns)

Index(['Launch Site', 'Lat', 'Long', 'class'], dtype='object')


In [12]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Crear el mapa base centrado en las coordenadas medias de los sitios de lanzamiento
m = folium.Map(location=[28.5721, -80.6480], zoom_start=5)

# Crear un MarkerCluster para agrupar los marcadores
marker_cluster = MarkerCluster().add_to(m)

# Iterar sobre cada fila de spacex_df y agregar un marcador para cada lanzamiento
for index, row in spacex_df.iterrows():
    # Definir las coordenadas y el color del marcador basado en el valor de 'class'
    coordinates = [row['Lat'], row['Long']]
    color = 'green' if row['class'] == 1 else 'red'
    
    # Crear el marcador
    folium.Marker(
        location=coordinates,
        icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0), html=f'<div style="font-size: 12px; color:{color};"><b>{row["Launch Site"]}</b></div>'),
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Mostrar el mapa con los marcadores
m


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Crear la nueva columna 'marker_color' basada en el valor de la columna 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verificar las primeras filas para asegurarte de que la columna se ha añadido correctamente
print(spacex_df.head())



   Launch Site        Lat       Long  class marker_color
0  CCAFS LC-40  28.562302 -80.577356      0          red
1  CCAFS LC-40  28.562302 -80.577356      0          red
2  CCAFS LC-40  28.562302 -80.577356      0          red
3  CCAFS LC-40  28.562302 -80.577356      0          red
4  CCAFS LC-40  28.562302 -80.577356      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
import folium
from folium.plugins import MarkerCluster

# Crear un mapa centrado en las coordenadas generales de los sitios de lanzamiento
m = folium.Map(location=[28.5721, -80.6480], zoom_start=5)

# Crear un MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Agregar los puntos existentes (verde y naranja) desde los datos previos
for index, row in spacex_df.iterrows():
    # Obtener las coordenadas del lanzamiento
    coordinates = [row['Lat'], row['Long']]
    
    # Obtener el color del marcador basado en la columna 'marker_color'
    color = row['marker_color']
    
    # Nombre del sitio de lanzamiento (puedes personalizar el nombre según necesites)
    site_name = f"Site{row['Launch Site'][-1]}"  # Asumimos que los nombres de los sitios terminan en número

    # Si la clase es 1 (éxito), marcamos el punto en verde, si es 0 (fallo), lo marcamos en naranja
    if color == 'green':
        folium.Marker(
            location=coordinates,
            popup=f"Launch Site: {site_name}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
            icon=folium.Icon(color='green')
        ).add_to(m)  # Agregamos el marcador verde

    elif color == 'orange':
        folium.Marker(
            location=coordinates,
            popup=f"Launch Site: {site_name}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
            icon=folium.Icon(color='orange')
        ).add_to(m)  # Agregamos el marcador naranja

# Ahora agregar los marcadores al MarkerCluster
for index, row in spacex_df.iterrows():
    # Obtener las coordenadas del lanzamiento
    coordinates = [row['Lat'], row['Long']]
    
    # Obtener el color del marcador basado en la columna 'marker_color'
    color = row['marker_color']
    
    # Nombre del sitio de lanzamiento (igual que arriba)
    site_name = f"Site{row['Launch Site'][-1]}"

    # Crear el marcador dentro del MarkerCluster
    folium.Marker(
        location=coordinates,
        popup=f"Launch Site: {site_name}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=color)  # Usar el color basado en el resultado
    ).add_to(marker_cluster)

# Mostrar el mapa
m




Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math

# Definir la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Devuelve la distancia en kilómetros

# Crear un nuevo DataFrame para almacenar las distancias
distances = []

# Iterar sobre cada sitio de lanzamiento
for index, row1 in spacex_df.iterrows():
    lat1, lon1 = row1['Lat'], row1['Long']
    site_name = row1['Launch Site']
    
    for index2, row2 in spacex_df.iterrows():
        lat2, lon2 = row2['Lat'], row2['Long']
        site_name2 = row2['Launch Site']
        
        # Calcular la distancia si no es el mismo sitio
        if site_name != site_name2:
            distance = haversine(lat1, lon1, lat2, lon2)
            distances.append((site_name, site_name2, distance))

# Crear un DataFrame con los resultados de las distancias
distances_df = pd.DataFrame(distances, columns=['Launch Site 1', 'Launch Site 2', 'Distance (km)'])

# Mostrar las primeras filas del DataFrame de distancias
print(distances_df)


     Launch Site 1 Launch Site 2  Distance (km)
0      CCAFS LC-40   VAFB SLC-4E    3825.840279
1      CCAFS LC-40   VAFB SLC-4E    3825.840279
2      CCAFS LC-40   VAFB SLC-4E    3825.840279
3      CCAFS LC-40   VAFB SLC-4E    3825.840279
4      CCAFS LC-40   VAFB SLC-4E    3825.840279
...            ...           ...            ...
2137  CCAFS SLC-40    KSC LC-39A       6.934099
2138  CCAFS SLC-40    KSC LC-39A       6.934099
2139  CCAFS SLC-40    KSC LC-39A       6.934099
2140  CCAFS SLC-40    KSC LC-39A       6.934099
2141  CCAFS SLC-40    KSC LC-39A       6.934099

[2142 rows x 3 columns]


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
from folium.features import DivIcon
import math

# Función Haversine para calcular la distancia
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Devuelve la distancia en kilómetros

# Coordenadas del sitio de lanzamiento (Ejemplo: CCAFS SLC-40)
launch_site_lat = 28.563197  # Ejemplo de latitud de un sitio de lanzamiento
launch_site_lon = -80.576820  # Ejemplo de longitud de un sitio de lanzamiento

# Coordenadas de la costa más cercana (Ejemplo)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcular la distancia entre el sitio de lanzamiento y la costa
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Crear un marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear un marcador para el punto en la costa más cercana
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Coastline",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(mymap)

# Mostrar la distancia entre el sitio de lanzamiento y la costa
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline)
    )
)

# Añadir el marcador de distancia al mapa
distance_marker.add_to(mymap)

# Mostrar el mapa
mymap


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [27]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
import folium

# Coordenadas del sitio de lanzamiento (Ejemplo: CCAFS SLC-40)
launch_site_lat = 28.563197  # Ejemplo de latitud de un sitio de lanzamiento
launch_site_lon = -80.576820  # Ejemplo de longitud de un sitio de lanzamiento

# Coordenadas de la costa más cercana (Ejemplo)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Crear un marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear un marcador para el punto en la costa más cercana
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Coastline",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(mymap)

# Crear una PolyLine entre el sitio de lanzamiento y la costa
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1, color="blue", opacity=0.7).add_to(mymap)

# Mostrar el mapa
mymap


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:


import folium
import math

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Resultado en kilómetros
    return distance

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.563197  # Latitud del sitio de lanzamiento
launch_site_lon = -80.576820  # Longitud del sitio de lanzamiento

# Coordenadas corregidas de la costa (hacia el este, aproximadamente a 0.90 km)
coast_lat = 28.563197  # Misma latitud que el sitio de lanzamiento
coast_lon = -80.567820  # Longitud ajustada hacia el este (0.9 km)

# Calcular la distancia entre el sitio de lanzamiento y la costa usando Haversine
distance_to_coast = haversine(launch_site_lat, launch_site_lon, coast_lat, coast_lon)

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Crear el marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear el marcador para la costa
folium.Marker(
    [coast_lat, coast_lon],
    popup=f"Coast (Distance: {distance_to_coast:.2f} km)",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(mymap)

# Crear una PolyLine entre el sitio de lanzamiento y la costa
coordinates = [[launch_site_lat, launch_site_lon], [coast_lat, coast_lon]]
folium.PolyLine(locations=coordinates, weight=2, color="blue", opacity=0.7).add_to(mymap)

# Mostrar el mapa
mymap


In [18]:
import folium
import math

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Resultado en kilómetros
    return distance

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordenadas de Samuel C. Philips Pkwy
samuel_pkwy_lat = 28.56342
samuel_pkwy_lon = -80.57080

# Calcular la distancia entre el sitio de lanzamiento y Samuel C. Philips Pkwy
distance_to_street = haversine(launch_site_lat, launch_site_lon, samuel_pkwy_lat, samuel_pkwy_lon)

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Crear el marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear el marcador para Samuel C. Philips Pkwy
folium.Marker(
    [samuel_pkwy_lat, samuel_pkwy_lon],
    popup=f"Samuel C. Philips Pkwy (Distance: {distance_to_street:.2f} km)",
    icon=folium.Icon(color="green", icon="road")
).add_to(mymap)

# Crear una PolyLine entre el sitio de lanzamiento y Samuel C. Philips Pkwy
coordinates = [[launch_site_lat, launch_site_lon], [samuel_pkwy_lat, samuel_pkwy_lon]]
folium.PolyLine(locations=coordinates, weight=2, color="red", opacity=0.8).add_to(mymap)

# Mostrar el mapa
mymap


In [19]:
import folium
import math

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Resultado en kilómetros
    return distance

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordenadas del ferrocarril de la NASA (NASA Railway)
nasa_railway_lat = 28.5721  # Ejemplo de latitud del ferrocarril
nasa_railway_lon = -80.5852  # Ejemplo de longitud del ferrocarril

# Calcular la distancia entre el sitio de lanzamiento y el ferrocarril
distance_to_railway = haversine(launch_site_lat, launch_site_lon, nasa_railway_lat, nasa_railway_lon)

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Crear el marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear el marcador para el ferrocarril de la NASA
folium.Marker(
    [nasa_railway_lat, nasa_railway_lon],
    popup=f"NASA Railway (Distance: {distance_to_railway:.2f} km)",
    icon=folium.Icon(color="purple", icon="train")
).add_to(mymap)

# Crear una PolyLine entre el sitio de lanzamiento y el ferrocarril
coordinates = [[launch_site_lat, launch_site_lon], [nasa_railway_lat, nasa_railway_lon]]
folium.PolyLine(locations=coordinates, weight=2, color="purple", opacity=0.8).add_to(mymap)

# Mostrar el mapa
mymap


In [21]:
import folium
import math

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Resultado en kilómetros
    return distance

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.563197  # Latitud del sitio de lanzamiento
launch_site_lon = -80.576820  # Longitud del sitio de lanzamiento

# Coordenadas del centro de Melbourne, FL
melbourne_lat = 28.0836  # Latitud de Melbourne
melbourne_lon = -80.6081  # Longitud de Melbourne

# Calcular la distancia entre el sitio de lanzamiento y Melbourne usando Haversine
distance_to_melbourne = haversine(launch_site_lat, launch_site_lon, melbourne_lat, melbourne_lon)

# Crear el mapa centrado en el sitio de lanzamiento
mymap = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Crear el marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(mymap)

# Crear el marcador para Melbourne
folium.Marker(
    [melbourne_lat, melbourne_lon],
    popup=f"Melbourne (Distance: {distance_to_melbourne:.2f} km)",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(mymap)

# Crear una PolyLine entre el sitio de lanzamiento y Melbourne
coordinates = [[launch_site_lat, launch_site_lon], [melbourne_lat, melbourne_lon]]
folium.PolyLine(locations=coordinates, weight=2, color="blue", opacity=0.7).add_to(mymap)

# Mostrar el mapa
mymap


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
